In [1]:
import json
from llama_models.llama3.reference_impl.generation import Llama
from llama_models.llama3.api.datatypes import UserMessage, SystemMessage
import os
import sys
from llama_models.llama3.api.datatypes import (
    UserMessage,
    SystemMessage,
    CompletionMessage,
    StopReason,
)
from llama_models.llama3.reference_impl.generation import Llama
import pandas as pd
from io import StringIO
import numpy as np
import re
import requests
import ollama
import random
from tiktoken import get_encoding
import time
import httpx


In [6]:
'''labels_with_definitions = [
    ("Peso", "Indica la ligereza de la zapatilla, generalmente expresado en gramos. El peso puede influir en el rendimiento y la comodidad durante la carrera."),
    ("Material del upper (parte superior)", "Describe los materiales utilizados en la parte superior de la zapatilla, como malla, cuero sintético o tejidos técnicos, que afectan la transpirabilidad, flexibilidad y soporte."),
    ("Material de la mediasuela", "Se refiere a los compuestos empleados en la entresuela, como espumas EVA o tecnologías propietarias, que proporcionan amortiguación y absorción de impactos."),
    ("Suela exterior", "Detalla el tipo de goma o caucho utilizado en la suela y el diseño del patrón de tracción, aspectos que influyen en el agarre y la durabilidad en diversas superficies."),
    ("Sistema de amortiguación", "Especifica las tecnologías o materiales destinados a reducir el impacto durante la pisada, contribuyendo al confort y la protección de las articulaciones."),
    ("Drop (diferencial talón-punta)", "Indica la diferencia de altura entre el talón y la punta de la zapatilla, medida en milímetros. Un drop alto suele ofrecer mayor amortiguación en el talón, mientras que un drop bajo promueve una pisada más natural."),
    ("Tipo de pisada", "Clasifica la zapatilla según su adecuación para diferentes tipos de pisada: neutra, pronadora o supinadora. Esto es esencial para elegir un calzado que se adapte a la biomecánica del corredor."),
    ("Tipo de uso", "Define el propósito principal de la zapatilla, como entrenamiento diario, competición, trail running o uso casual, lo que orienta sobre su diseño y funcionalidades específicas."),
    ("Género", "Indica si la zapatilla está diseñada para hombres, mujeres o es un modelo unisex, considerando diferencias anatómicas y de tamaño."),
    ("Tallas disponibles", "Especifica el rango de tallas en las que se ofrece la zapatilla, asegurando que el corredor pueda encontrar un ajuste adecuado."),
    ("Anchura", "Algunas marcas ofrecen diferentes anchos (estrecho, estándar, ancho) para adaptarse a diversas morfologías del pie."),
    ("Precio", "Proporciona el costo de la zapatilla, un factor determinante en la decisión de compra."),
    ("Tecnologías adicionales", "Incluye características especiales como impermeabilidad, reflectividad, sistemas de ajuste personalizados o elementos de estabilidad que mejoran la funcionalidad del calzado."),
]'''

'labels_with_definitions = [\n    ("Peso", "Indica la ligereza de la zapatilla, generalmente expresado en gramos. El peso puede influir en el rendimiento y la comodidad durante la carrera."),\n    ("Material del upper (parte superior)", "Describe los materiales utilizados en la parte superior de la zapatilla, como malla, cuero sintético o tejidos técnicos, que afectan la transpirabilidad, flexibilidad y soporte."),\n    ("Material de la mediasuela", "Se refiere a los compuestos empleados en la entresuela, como espumas EVA o tecnologías propietarias, que proporcionan amortiguación y absorción de impactos."),\n    ("Suela exterior", "Detalla el tipo de goma o caucho utilizado en la suela y el diseño del patrón de tracción, aspectos que influyen en el agarre y la durabilidad en diversas superficies."),\n    ("Sistema de amortiguación", "Especifica las tecnologías o materiales destinados a reducir el impacto durante la pisada, contribuyendo al confort y la protección de las articulaciones.

In [ ]:
labels_with_definitions = [
    ("Weight", "Indicates the lightness of the shoe, usually expressed in grams. Weight can influence performance and comfort during running."),
    ("Upper Material", "Describes the materials used in the shoe's upper part, such as mesh, synthetic leather, or technical fabrics, which affect breathability, flexibility, and support."),
    ("Midsole Material", "Refers to the compounds used in the midsole, such as EVA foams or proprietary technologies, which provide cushioning and shock absorption."),
    ("Outsole", "Details the type of rubber or material used in the sole and the traction pattern design, which influence grip and durability on various surfaces."),
    ("Cushioning System", "Specifies the technologies or materials aimed at reducing impact during strides, contributing to comfort and joint protection."),
    ("Drop (heel-to-toe differential)", "Indicates the height difference between the heel and the toe of the shoe, measured in millimeters. A higher drop typically offers more heel cushioning, while a lower drop promotes a more natural stride."),
    ("Pronation Type", "Classifies the shoe according to its suitability for different pronation types: neutral, overpronation, or supination. This is essential for choosing footwear that matches the runner's biomechanics."),
    ("Usage Type", "Defines the primary purpose of the shoe, such as daily training, racing, trail running, or casual use, guiding its specific design and features."),
    ("Gender", "Indicates whether the shoe is designed for men, women, or is a unisex model, considering anatomical and sizing differences."),
    ("Available Sizes", "Specifies the range of sizes in which the shoe is offered, ensuring the runner can find a suitable fit."),
    ("Width", "Some brands offer different widths (narrow, standard, wide) to accommodate various foot shapes."),
    ("Additional Technologies", "Includes special features such as waterproofing, reflectivity, customized fit systems, or stability elements that enhance the shoe's functionality."),
]


## Funciones

In [ ]:


def remove_html_tags(text):
    if text:
        text = re.sub(r"<.*?>", "", text)  # Remueve etiquetas HTML
        text = text.replace('&nbsp;', ' ')
        text = text.strip()
    return text

def transformar_texto(texto, marca):
    if texto is None or (isinstance(texto, float) and np.isnan(texto)):
        return texto

    if marca.lower() == "adidas":
        # Transformación original para adidas
        if isinstance(texto, (list, np.ndarray)):
            texto = ", ".join(map(str, texto))
        else:
            texto = str(texto)
        texto = texto.strip("[]")
        texto = re.sub(r",\s*", '} {', texto)
        texto = '{' + texto + '}'
        return texto

    elif marca.lower() == "nike":
        if isinstance(texto, list):
            # Si es lista de diccionarios (details)
            if all(isinstance(item, dict) for item in texto):
                texto_limpio = []
                for d in texto:
                    if isinstance(d, dict):
                        d_limpio = {}
                        for k, v in d.items():
                            k_clean = k.strip() if k else ""
                            v_clean = remove_html_tags(v) if isinstance(v, str) else v
                            # Si clave y valor no están vacíos
                            if k_clean and v_clean:
                                # Aquí verificamos si la info es irrelevante: 
                                # Caso en el que el valor es igual a la clave (sin html), 
                                # Ejemplo: { "Datos del producto": "<b>Datos del producto</b>" }
                                # No aporta información adicional, así que no la incluimos.
                                if v_clean != k_clean:
                                    d_limpio[k_clean] = v_clean

                        if d_limpio:
                            texto_limpio.append(d_limpio)

                # Si después de limpiar no quedó nada (porque todo era texto irrelevante),
                # devolvemos lista vacía
                return texto_limpio

            else:
                # Si es lista de strings (characteristics)
                seen = set()
                texto_limpio = []
                for s in texto:
                    if s and isinstance(s, str):
                        s_clean = remove_html_tags(s)
                        if s_clean and s_clean not in seen:
                            seen.add(s_clean)
                            texto_limpio.append(s_clean)
                return texto_limpio

        else:
            # Si no es lista (solo string)
            if isinstance(texto, str):
                return remove_html_tags(texto)
            return texto

    else:
        raise ValueError(f"Marca '{marca}' no reconocida. Usa 'adidas' o 'nike'.")


def obtener_respuesta_ollama(prompt, max_retries=3, delay=5):
    attempt = 0
    while attempt < max_retries:
        try:
            response = ollama.chat(
                model="llama3.2:latest",
                messages = [
                   {
                       "role":"user",
                       "content": prompt
                   } 
                ]
            )
            # Si llega aquí, la respuesta se obtuvo exitosamente
            return response
        except httpx.HTTPError as e:
            attempt += 1
            print(f"Error en la solicitud: {e}. Reintento {attempt} de {max_retries} en {delay} segundos...")
            time.sleep(delay)
    
    # Si sale del while sin retornar, significa que falló en todos los intentos
    raise RuntimeError(f"No se pudo obtener respuesta de Ollama después de {max_retries} intentos.")







def generate_prompt_ollama(product, labels_with_definitions):
    labels = [label for label, _ in labels_with_definitions]
    prompt = f"""
        You are an assistant specialized in extracting structured information from product descriptions and organizing it into tables.
        Your task is to extract the following information from the product details and label it according to the provided labels: {', '.join(labels)}.
        Each label has the following definition to help guide your extraction:

        {''.join([f'- {label}: {definition}\n' for label, definition in labels_with_definitions])}

        If a label does not have a clear match in the details, complete its value with "null".

        Product information:
        - Details: {product['details']}
        - Description: {product['description']}
        - Category: {product['category']}
        - Characteristics: {product['characteristics']}

        Provide the information in a table with columns corresponding to each label. 
        The table must include **two complete rows**:
        1. The first row contains the label names.
        2. The second row contains the corresponding labeled values.

        Expected response format:
        | {' | '.join(labels)} |
        | {' | '.join(['---'] * len(labels))} |
        | value_1 | value_2 | ... |
        
        Example:
        If the labels are "details", "description", and "category", and the extracted values are 
        "Comfortable sneakers", "Made with recycled materials", and "Footwear", respectively, 
        your response should be:

        | details | description | category |
        | --- | --- | --- |
        | Comfortable sneakers | Made with recycled materials | Footwear |

        Remember:
        - The response must contain two complete rows.
        - Only respond with the table and **do not include additional text**.

        Now, extract and structure the information for the provided product:

        | {' | '.join(labels)} |
        | {' | '.join(['---'] * len(labels))} |
        |"""
    return prompt.strip()

# Tokenizer function (using tiktoken for GPT-like models)
def count_tokens(text):
    try:
        encoding = get_encoding("cl100k_base")  # Example encoding for GPT-like models
        return len(encoding.encode(text))
    except Exception:
        return len(text.split())  # Fallback: approximate by word count
# Función de simulación de Monte Carlo corregida
def monte_carlo_simulation(products, models, iterations=1000, num_products=None):
    results = {}
    
    for model_name, model_info in models.items():
        tokens_per_product = {}
        costs_per_product = {}
        
        for _ in range(iterations):
            # Muestra una fracción de los productos si se especifica
            if num_products is not None and num_products < len(products):
                sampled_products = random.sample(products, num_products)
            else:
                sampled_products = products

            for product in sampled_products:
                product_id = product.get('id', id(product))  # Usamos un identificador único para cada producto
                prompt = generate_prompt_ollama(product, labels_with_definitions)
                tokens = count_tokens(prompt)
                
                # Verifica si el número de tokens excede la ventana de contexto
                if tokens > model_info['context_window']*1000:
                    print(f"Advertencia: El prompt excede la ventana de contexto para el modelo {model_name}")
                    # Puedes manejar este caso según necesites (e.g., omitir, truncar)
                    continue
                
                cost = (tokens / 1000) * model_info['cost_in']
                
                # Actualiza los máximos y mínimos por producto
                if product_id not in tokens_per_product:
                    tokens_per_product[product_id] = {'max': tokens, 'min': tokens}
                    costs_per_product[product_id] = {'max': cost, 'min': cost}
                else:
                    tokens_per_product[product_id]['max'] = max(tokens_per_product[product_id]['max'], tokens)
                    tokens_per_product[product_id]['min'] = min(tokens_per_product[product_id]['min'], tokens)
                    costs_per_product[product_id]['max'] = max(costs_per_product[product_id]['max'], cost)
                    costs_per_product[product_id]['min'] = min(costs_per_product[product_id]['min'], cost)
        
        # Después de todas las iteraciones, obtenemos los máximos y mínimos globales
        max_tokens = max([data['max'] for data in tokens_per_product.values()], default=0)
        min_tokens = min([data['min'] for data in tokens_per_product.values()], default=0)
        max_cost = max([data['max'] for data in costs_per_product.values()], default=0)
        min_cost = min([data['min'] for data in costs_per_product.values()], default=0)
        
        results[model_name] = {
            'max_tokens': max_tokens,
            'min_tokens': min_tokens,
            'max_cost': max_cost,
            'min_cost': min_cost,
        }
    return results

In [94]:
def procesar_respuesta(respuesta_texto, etiquetas):
    try:
        # Buscar el inicio de la tabla
        inicio_tabla = respuesta_texto.find('|')
        if inicio_tabla == -1:
            print("No se encontró una tabla en la respuesta.")
            return None

        # Extraer la tabla desde el primer '|'
        tabla_texto = respuesta_texto[inicio_tabla:].strip()
        # Extraer solo las líneas que contengan '|'
        lineas = tabla_texto.split('\n')
        lineas_tabla = []
        for linea in lineas:
            if '|' in linea:
                # Eliminar los '|' iniciales y finales y espacios extra
                linea = linea.strip().strip('|').strip()
                lineas_tabla.append(linea)
            else:
                # Detenerse si la línea no contiene '|'
                break

        if not lineas_tabla:
            print("No se encontraron líneas válidas de tabla.")
            return None

        # Verificar si hay separadores '---'
        tiene_separador = any(
            all(c.strip('-') == '' for c in fila.split('|')) 
            for fila in lineas_tabla
        )

        # Caso 1: Hay separadores (modo normal de Markdown)
        if tiene_separador:
            tabla_completa = '\n'.join(lineas_tabla)
            df = pd.read_csv(StringIO(tabla_completa), sep='|', engine='python', skipinitialspace=True)
            # Limpiar columnas y datos
            df.columns = [col.strip() for col in df.columns]
            for col in df.columns:
                if df[col].dtype == object:
                    df[col] = df[col].str.strip()
            df = df.reset_index(drop=True)
            return df

        # Caso 2: No hay separadores
        # Verificar la cantidad de líneas
        if len(lineas_tabla) == 2:
            # La primera línea son los nombres de columnas, la segunda línea datos
            columnas = [c.strip() for c in lineas_tabla[0].split('|')]
            datos = [c.strip() for c in lineas_tabla[1].split('|')]
            df = pd.DataFrame([datos], columns=columnas)
            for col in df.columns:
                if df[col].dtype == object:
                    df[col] = df[col].str.strip()
            df = df.reset_index(drop=True)
            return df
        elif len(lineas_tabla) == 1:
            # Solo una línea de datos, usar etiquetas como nombres de columnas
            datos = [c.strip() for c in lineas_tabla[0].split('|')]
            # Ajustar si el número de columnas difiere del número de etiquetas
            if len(etiquetas) != len(datos):
                print("Advertencia: El número de etiquetas no coincide con las columnas detectadas. Se ajustará al mínimo.")
                col_names = etiquetas[:len(datos)]
            else:
                col_names = etiquetas

            df = pd.DataFrame([datos], columns=col_names)
            for col in df.columns:
                if df[col].dtype == object:
                    df[col] = df[col].str.strip()
            df = df.reset_index(drop=True)
            return df
        else:
            # Hay más de 2 líneas pero sin separador. Asumimos que la primera es columnas y el resto datos.
            columnas = [c.strip() for c in lineas_tabla[0].split('|')]
            filas_datos = [ [x.strip() for x in fila.split('|')] for fila in lineas_tabla[1:] ]
            df = pd.DataFrame(filas_datos, columns=columnas)
            for col in df.columns:
                if df[col].dtype == object:
                    df[col] = df[col].str.strip()
            df = df.reset_index(drop=True)
            return df

    except Exception as e:
        print(f"Error al procesar la tabla: {e}")
        return None


## Lectura de datos

In [5]:
url ="https://scraping-firestore-178159629911.us-central1.run.app//v1/scraping/"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(response.status_code)
else:
    print(f'Error: {response.status_code}')

df_raw = pd.DataFrame(data)

200


## Clasificacion

In [6]:
df_adidas = df_raw[df_raw['store'] == 'adidas']

In [7]:
# Lista de descripciones de productos
df_adidas['details_transformado'] = df_adidas['details'].apply(lambda x: transformar_texto(x, 'adidas'))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2256\354877031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adidas['details_transformado'] = df_adidas['details'].apply(lambda x: transformar_texto(x, 'adidas'))


In [83]:
# Crear un diccionario con las llaves deseadas desde todo el DataFrame
productos = [
    {
        "id": row['id'],
        "regularPrice" : row["regularPrice"],
        "undiscounted_price": row["undiscounted_price"],
        "details": row['details_transformado'],
        "description": row['description'],
        "category": row['category'],
        "characteristics": row['characteristics']
    }
    for _, row in df_adidas.iterrows()  # Recorre todo el DataFrame
    if row['details_transformado'] != '{}'  # Filtra donde los detalles no estén vacíos
]

In [99]:
dfs = []
for producto in productos[1:10]:
    prompt = generate_prompt_ollama(producto, labels_with_definitions)
    respuesta = obtener_respuesta_ollama(prompt)
    print(producto)
    print("Respuesta del modelo:")
    print(respuesta["message"]["content"])
    print("\n------------------------\n")
    df = procesar_respuesta(respuesta["message"]["content"], labels_with_definitions)
    if df is not None:
        attribute_columns = df.columns[:-3]
        df['id'] = producto['id']
        df['regularPrice'] = producto['regularPrice']
        df['undiscounted_price'] = producto['undiscounted_price']
        df["details"] = producto['details']
        df["description"] = producto['description']
        df["category"] = producto['category']
        df["characteristics"] = producto['characteristics']
        df = df[~df[attribute_columns].eq('---').all(axis=1)]
        df = df.dropna(how='all')
        dfs.append(df)
        
    else:
        print("No se pudo extraer la tabla.\n")

{'id': '0AqheRhKT2lhm7puBVCF', 'regularPrice': '$799.950', 'undiscounted_price': 'NA', 'details': '{Ajuste clásico} {Sistema de amarre de cordones} {Exterior de tejido adidas Primeknit} {Ajuste suave en el talón} {Sistema Linear Energy Push} {Mediasuela BOOST} {Peso: 289 g (Talla CO 37)} {Suela Stretchweb con caucho Continental™ Better Rubber} {El hilo del exterior contiene al menos un 50% de Parley Ocean Plastic y un 50% de poliéster reciclado} {Color del artículo: Core Black / Core Black / Cloud White} {Número de artículo: GX5591}', 'description': 'Hemos analizado 1.200.000 pisadas para que Ultraboost evolucione y mejore su ajuste para adaptarse 360° al pie femenino. Y aún hay más. Hemos rediseñado la suela de caucho. Probamos cientos de prototipos. Hasta que comprobamos las mejoras en su rendimiento. ¿El resultado? Un 4 % más de energía que los Ultraboost 21 para mujer. El ajuste en el área del talón del exterior adidas PRIMEKNIT se diseñó para evitar que el talón se deslice y se fo

In [102]:
if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
else:
    print("No se pudo crear el DataFrame total.")

In [103]:
df_total

,Weight,Upper Material,Midsole Material,Outsole,Cushioning System,Drop (heel-to-toe differential),Pronation Type,Usage Type,Gender,Available Sizes,Width,Additional Technologies,id,regularPrice,undiscounted_price,details,description,category,characteristics
0,289 g,adidas Primeknit,Mediasuela BOOST,Suela Stretchweb con caucho Continental Better...,Sistema Linear Energy Push,null,neutral,mujer,37-47,standard,contains Parley Ocean Plastic and recycled pol...,None,0AqheRhKT2lhm7puBVCF,$799.950,NA,{Ajuste clásico} {Sistema de amarre de cordone...,Hemos analizado 1.200.000 pisadas para que Ult...,Mujer • Running,NaN
1,166 g,Parte superior de malla técnica,Amortiguación Lightstrike Pro,Suela de caucho Continental™,Amortiguación Lightstrike Pro,6 mm (talón: 32 mm / antepié: 26 mm),Neutral,"Daily training, racing, trail running, casual use",Mujer,"Talla COL 36 1/2, COL 37, COL 38, COL 39.5","Narrow, Standard, Wide",Contiene al menos un 20 % de material reciclado,0IgYTzUHkE7zIdcVyFCK,$1.049.950,$629.970,{Ajuste clásico} {Cierre de cordones} {Parte s...,Haz tus mejores 10k con nuestros nuevos tenis ...,Mujer • Running,NaN
2,200 gramos,Parte superior de malla,Amortiguación Lightstrike Pro,Suela de caucho Continental™ Rubber,Amortiguación Lightstrike Pro,6 mm (talón: 33 mm / antepié: 27 mm),null,Trail running,Hombre,"CO 40, CO 42-46, CO 48",null,Contienen al menos un 20% de material reciclado,0MU8aKCnCUZv2r9aLD67,$1.049.950,$734.965,{Ajuste clásico} {Sistema de amarre de cordone...,Haz tus mejores 10k con nuestros nuevos tenis ...,Hombre • Running,NaN
3,null,Sistema de amarre de cordones,ENERGYRODS 2.0,null,Lightstrike Pro,null,null,null,null,null,null,Supera tu marca personal y llega a la meta más...,0MU8aKCnCUZv2r9aLD67,$1.049.950,$734.965,{Ajuste clásico} {Sistema de amarre de cordone...,Haz tus mejores 10k con nuestros nuevos tenis ...,Hombre • Running,NaN
4,319 g,Parte superior textil,Mediasuela Cloudfoam,Suela de TPU,null,6 mm (talón 35 mm / antepié 29 mm),Neutral,Daily training,Men,"CO 40, CO 41-45, CO 46-50","Narrow, Standard, Wide",null,0Q6DNSlvsjBzy3AQeY2y,$279.950,NA,{Horma clásica} {Sistema de amarre de cordones...,Cada carrera es un viaje de descubrimiento. Po...,Hombre • Running,NaN
5,null,{Si este artículo es personalizado} / {Sistema...,{Amortiguación Lightstrike y Lightstrike Pro},{Suela de caucho},null,8 mm,neutral,"Trail running, casual use","Women, Men",5-12,null,El exterior contiene al menos un 50% de materi...,0SF7zveew5mzdUJWZKyz,$699.950,$559.960,{Si este artículo es personalizado} {no aplica...,"Cuando se trata de alcanzar tus metas, cada se...",Mujer • Running,NaN
6,304 g,"Parte superior de malla, Forro interno textil",Plantilla OrthoLite®,Suela Adiwear,Mediasuela Cloudfoam,10 mm (talón: 33 mm / antepié: 23 mm),null,Running,Hombre • Hombre,CO 40 - IE 8818,null,"Waterproofing, Reflectivity",0iPjAsLy8yEYvGgiCAzo,$299.950,NA,{Horma clásica} {Sistema de amarre de cordones...,"Tanto en la pista como en la cinta de correr, ...",Hombre • Running,NaN
7,213 gramos,"Parte superior de malla, Forro interno textil,...",mediasuela Dreamstrike+,Suela Adiwear,mediasuela Dreamstrike+,10 mm (talón: 34 mm / 24 mm),Neutral,Running,Mujer,CO 37,null,Contienen al menos un 20% de material reciclad...,0n2Tyl34QdVdkCKgAdcT,$649.950,NA,{Ajuste clásico} {Sistema de amarre de cordone...,Avanza hacia tus metas de running con estos te...,Mujer • Running,NaN
8,null,Exterior de malla con cuello acolchado / {Sist...,Mediasuela Bounce,Suela sintética,mediasuela Bounce,null,neutral,Mujer • Running,null,null,Legend Ink / Wonder Clay,IG0334,0rf6HEvBi5R4FunIhToR,$449.950,NA,{Ajuste clásico} {Sistema de amarre de cordone...,Estos tenis fueron diseñados para darte la amo...,Mujer • Running,NaN
9,334 gramos,"Exterior de malla, Tejido suave y cómodo",Mediasuela REPETITOR y REPETITOR+ de doble den...,Suela de caucho,mediasuela REPETITOR y REPETITOR+,6 mm (talón: 38 mm / antepié: 32 mm),Neutral,Daily training,Men,"CO 40-50, CO 42-44, CO 46","Narrow, Standard

In [105]:
output_path = "../src/comparative_analysis/database/adidas_etiquetado_llama31.xlsx"
df_total.to_excel(output_path, index=False)

## Preprocesamiento Nike

In [21]:
df_nike = df_raw[df_raw['store'] == 'nike'] 


In [60]:
# Lista de descripciones de productos
df_nike['details_transformado'] = df_nike['details'].apply(lambda x: transformar_texto(x, 'nike'))
df_nike['characteristics_transformado'] = df_nike['characteristics'].apply(lambda x: transformar_texto(x, 'nike'))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2256\2520405701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nike['details_transformado'] = df_nike['details'].apply(lambda x: transformar_texto(x, 'nike'))
C:\Users\Usuario\AppData\Local\Temp\ipykernel_2256\2520405701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nike['characteristics_transformado'] = df_nike['characteristics'].apply(lambda x: transformar_texto(x, 'nike'))


In [71]:
# Crear un diccionario con las llaves deseadas desde todo el DataFrame
productos_nike = [
    {
        "id": row['id'],
        "regularPrice" : row["regularPrice"],
        "undiscounted_price": row["undiscounted_price"],
        "details": row['details_transformado'],
        "description": row['description'],
        "category": row['category'],
        "characteristics": row['characteristics_transformado']
    }
    for _, row in df_nike.iterrows()  # Recorre todo el DataFrame
    if row['details_transformado'] != '{}'  # Filtra donde los detalles no estén vacíos
]

## Preprocesamiento nacion runner

In [73]:
df_nr = df_raw[df_raw['store'] == 'nacionRunner']

In [82]:
# Crear un diccionario con las llaves deseadas desde todo el DataFrame
productos_nr = [
    {
        "id": row['id'],
        "regularPrice" : row["regularPrice"],
        "undiscounted_price": row["undiscounted_price"],
        "details": row['details'],
        "description": row['description'],
        "category": row['category'],
        "characteristics": row['characteristics']
    }
    for _, row in df_nr.iterrows()  # Recorre todo el DataFrame
    if row['details'] != '{}'  # Filtra donde los detalles no estén vacíos
]